In [2]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import tensorflow as tf

# Define paths and parameters
train_dir = 'training_data'
test_dir = 'testing_Data'
image_height = 32
image_width = 32
batch_size = 32

# Create data generator
datagen = ImageDataGenerator(rescale=1./255)

test_dataset = datagen.flow_from_directory(
    test_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False
)

# Load the trained CNN model
cnn_model = load_model('cnn_improved.keras')

# Load the trained SVM model
svm_model = joblib.load('svm_model.joblib')

# Load the pre-trained VGG16 model for feature extraction
base_model = tf.keras.applications.VGG16(include_top=False, input_shape=(image_height, image_width, 3))
base_model.trainable = False

# Function to extract features using the pre-trained model
def extract_features(dataset, model):
    features = model.predict(dataset)
    features = features.reshape((features.shape[0], -1))  # Flatten the features
    return features

# Extract features for the test dataset using the base model
test_features = extract_features(test_dataset, base_model)

# Get predictions from CNN
cnn_pred_proba = cnn_model.predict(test_dataset)
cnn_pred = np.argmax(cnn_pred_proba, axis=-1)

# Get predictions from SVM
svm_pred = svm_model.predict(test_features)

# Get true labels
y_true = test_dataset.classes

# Combine the predictions into a single feature set
X_meta = np.column_stack((cnn_pred, svm_pred))

# Define the meta-model
meta_model = LogisticRegression()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'lbfgs']
}

# Create the GridSearchCV object
grid_search_meta = GridSearchCV(estimator=meta_model, param_grid=param_grid, 
                                scoring='precision_weighted', cv=5, n_jobs=-1, verbose=2)

# Fit the grid search to the data
grid_search_meta.fit(X_meta, y_true)

# Get the best parameters and best meta-model
best_meta_params = grid_search_meta.best_params_
best_meta_model = grid_search_meta.best_estimator_

# Predict using the meta-model
y_meta_pred = best_meta_model.predict(X_meta)

# Calculate metrics
accuracy = accuracy_score(y_true, y_meta_pred)
precision = precision_score(y_true, y_meta_pred, average='weighted')
recall = recall_score(y_true, y_meta_pred, average='weighted')
f1 = f1_score(y_true, y_meta_pred, average='weighted')
conf_matrix = confusion_matrix(y_true, y_meta_pred)

# Print the metrics
print("Confusion Matrix (Meta-Model):")
print(conf_matrix)
print("\nMetrics (Meta-Model):")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Print the best parameters
print("Best Parameters found by Grid Search:")
print(best_meta_params)

# Save the meta-model
joblib.dump(best_meta_model, 'best_meta_model.joblib')


Found 1008 images belonging to 36 classes.


c:\Users\Ashad InnO\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 143ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Confusion Matrix (Meta-Model):
[[20  7  0 ...  0  0  0]
 [ 2 26  0 ...  0  0  0]
 [ 0  0 28 ...  0  0  0]
 ...
 [ 0  0  0 ... 28  0  0]
 [ 0  0  0 ...  0  0 28]
 [ 0  0  0 ...  0  0 28]]

Metrics (Meta-Model):
Accuracy: 0.8283730158730159
Precision: 0.7933358660503043
Recall: 0.8283730158730159
F1 Score: 0.7810851172858926
Best Parameters found by Grid Search:
{'C': 100, 'solver': 'liblinear'}


c:\Users\Ashad InnO\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


['best_meta_model.joblib']